In [ ]:
import torch
import pandas as pd
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob
from format_data import *

In [ ]:
folder_name = "models/diabetes"+"/generator*.pt"
model_widget = widgets.Dropdown(
    options=glob.glob(folder_name),
    description='Generator:',
    disabled=False,
)
display(model_widget)

In [ ]:
original_db_name = "models/diabetes"[7:]
original_db_path = "original_data/" + original_db_name + ".csv"
original_db = pd.read_csv(original_db_path)
original_db_size=original_db.shape[0]

In [ ]:
try:
    checkpoint= torch.load(model_widget.value, map_location='cuda')
except:
    checkpoint= torch.load(model_widget.value, map_location='cpu')
checkpoint['model_attributes']['out_features'] = len(original_db.columns)
generator = GeneratorNet(**checkpoint['model_attributes'])
generator.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
size = original_db_size
new_data = generator.create_data(size)
df = pd.DataFrame(new_data, columns=original_db.columns)
#Changes the name to be easier to read
name = model_widget.value.split("/")[-1][9:-4] + "_size-" + str(size)
df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)

In [ ]:
#Do the same thing as the cells above but for all the files in the directory
import glob
for file in glob.glob(folder_name):
    name = file.split("/")[-1][9:-4]
    print(name)
    try:
        checkpoint= torch.load(file, map_location='cuda')
    except:
        checkpoint= torch.load(file, map_location='cpu')
    generator = GeneratorNet(**checkpoint['model_attributes'])
    generator.load_state_dict(checkpoint['model_state_dict'])
    size = original_db_size
    new_data = generator.create_data(size)
    new_data = format_output(new_data)
    df = pd.DataFrame(new_data, columns=original_db.columns)
    df = format_output_db(df)
    name = name + "_size-" + str(size)
    df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)